In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import copy
import datetime
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')

from utils.data_extraction import load_data
from utils.data_cleaning import HFDataCleaning
from utils.generate_features import candleCreateNP_vect_final, generateFeatures_final
from utils.preprocessing_features_and_labels import extract_labels

### Load data etc

In [2]:
# print(os.listdir())
try:
    path = 'a:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
except:
    path = 't:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)    
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,32)]).astype(int)
dates = np.array(['20200401','20200402']).astype(int)

# Provide a list of tickers of interest
tickers = ['GOOG']#'MSFT'

# Do we need data on trades, quotes or both?
dataNeeded = 'trades' # 'trades', 'quotes' or 'both'

# run load_data()
if dataNeeded == 'trades':
    tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'quotes':
    quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'both':
    tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

['.ipynb_checkpoints', 'Old Versions', 'preprocessing.ipynb', 'testing_and_comparing_candlesticks.ipynb', 'testing_features.ipynb', 'testing_preprocessing_features_and_labels.ipynb']
8515 ['taq_19930315.h5', 'taq_19930104.h5', 'taq_19930317.h5', 'taq_19930105.h5', 'taq_19930316.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200401
Date, Max: 20200402

##### Data Extraction begins #####

trade data is being extracted..

### Trade Data ###

The raw H5 trade file contains:  ['TradeIndex', 'Trades'] 

Ticker Information:  (b'GOOG            ', 26900500, 71427) 

Sneak peak of the data

            utcsec ex  cond  volume    price TradeStop

In [3]:
DATA_SAMPLE = 'full' # or 'stable'

if DATA_SAMPLE == 'stable':
    # P1 is used for keeping data within [9.5, 16]
    cleanedData = HFDataCleaning(['P1','p2','t1','p3'],tradeData,'trade',['q','d'])
elif DATA_SAMPLE == 'full':
    # P1_2 is used for keeping data within [9, 16.5]
    cleanedData = HFDataCleaning(['P1_2','p2', 't1', 'p3'],tradeData,'trade',['q','d'])    

In [4]:
tradeData[tradeData.Timestamp.dt.hour == 9] 

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,TradeReportingFacility,ParticipantTime,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker
877,32402041512373,D,@ TI,30,1124.4600,,00,25795,23,N,N,32402038000000,32402041146104,0,20200401,2020-04-01 09:00:02.041512,373,9,0,GOOG
878,32406408120050,K,@ TI,2,1125.7000,,00,25808,257,N,,32406407880000,99,0,20200401,2020-04-01 09:00:06.408120,050,9,0,GOOG
879,32406409053327,K,@ TI,1,1125.5000,,00,25809,258,N,,32406408841000,99,0,20200401,2020-04-01 09:00:06.409053,327,9,0,GOOG
880,32406409838624,K,@ TI,4,1125.0000,,00,25810,259,N,,32406409633000,99,0,20200401,2020-04-01 09:00:06.409838,624,9,0,GOOG
881,32406410609648,K,@ TI,1,1124.5000,,00,25811,260,N,,32406410405000,99,0,20200401,2020-04-01 09:00:06.410609,648,9,0,GOOG
882,32406410616176,K,@ TI,22,1124.5000,,00,25812,261,N,,32406410405000,99,0,20200401,2020-04-01 09:00:06.410616,176,9,0,GOOG
883,32406489411475,P,@ TI,30,1124.5100,,00,25822,250,N,,32406489037312,99,0,20200401,2020-04-01 09:00:06.489411,475,9,0,GOOG
884,32409065024949,P,@ TI,15,1124.5100,,00,25865,251,N,,32409064655104,99,0,20200401,2020-04-01 09:00:09.065024,949,9,0,GOOG
885,32409084256614,P,@ TI,15,1124.5100,,00,25869,252,N,,32409083889152,99,0,20200401,2020-04-01 09:00:09.084256,614,9,0,GOOG
886,32409466901817,P,@ TI,15,1124.5100,,00,25890,253,N,,32409466531072,99,0,20200401,2020-04-01 09:00:09.466901,817,9,0,GOOG


In [5]:
tradeData[(tradeData.Timestamp.dt.hour == 16) & (tradeData.Timestamp.dt.minute == 29)] 
# no data from exchange Q after 16:22:30

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,TradeReportingFacility,ParticipantTime,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker
71179,59359931498154,D,@ TI,21,1105.62,,00,4328878,16774,N,Q,59359924000000,59359931442725,0,20200401,2020-04-01 16:29:19.931498,154,16,29,GOOG
56472,59377688654669,D,@ TI,1,1119.00,,00,4768036,3413,N,N,59377685000000,59377688294896,0,20200402,2020-04-02 16:29:37.688654,669,16,29,GOOG


In [6]:
cleanedData

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,TradeReportingFacility,ParticipantTime,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker
0,32402041512373,D,@ TI,30,1124.4600,,00,25795,23,N,N,32402038000000,32402041146104,0,20200401,2020-04-01 09:00:02.041512,373,9,0,GOOG
1,32444512198659,D,@ TI,6,1124.2033,,00,26270,24,N,N,32444509000000,32444511840333,0,20200401,2020-04-01 09:00:44.512198,659,9,0,GOOG
2,32485885409069,Q,@ TI,30,1124.6200,,00,26476,293,N,,32485885387917,99,0,20200401,2020-04-01 09:01:25.885409,069,9,1,GOOG
3,32486612971716,Q,@ TI,30,1124.5100,,00,26492,294,N,,32486612950358,99,0,20200401,2020-04-01 09:01:26.612971,716,9,1,GOOG
4,32519261072977,Q,@ TI,1,1125.7400,,00,26668,295,N,,32519261050258,99,0,20200401,2020-04-01 09:01:59.261072,977,9,1,GOOG
5,32525016359664,Q,@ TI,2,1125.7400,,00,26710,296,N,,32525016338305,99,0,20200401,2020-04-01 09:02:05.016359,664,9,2,GOOG
6,32531700656053,D,@ TI,40,1124.2000,,00,26753,44,N,Q,32531698000000,32531700616806,0,20200401,2020-04-01 09:02:11.700656,053,9,2,GOOG
7,32601332718939,D,@ TI,1,1124.2000,,00,27141,25,N,N,32601330000000,32601332358257,0,20200401,2020-04-01 09:03:21.332718,939,9,3,GOOG
8,32611276175419,Q,@ TI,27,1125.2200,,00,27229,297,N,,32611276152332,99,0,20200401,2020-04-01 09:03:31.276175,419,9,3,GOOG
9,32611276179601,Q,@ TI,2,1125.4000,,00,27230,298,N,,32611276152332,99,0,20200401,2020-04-01 09:03:31.276179,601,9,3,GOOG


In [7]:
# the first bar is NaN throughout. It's not an error in the candlesticking, but rather it's due to the P3 cleaning which removes all data points between 09:00:00 and 09:01:00 on 2020-04-01
# so apparently Google has no trades on Nasdaq before 1 minute into the open
candles = candleCreateNP_vect_final(cleanedData, step=1, sample=DATA_SAMPLE, numpied=True)
candles

array([[1124.46  , 1124.46  , 1124.2033, 1124.2033],
       [1124.62  , 1125.74  , 1124.51  , 1125.74  ],
       [1125.74  , 1125.74  , 1124.2   , 1124.2   ],
       ...,
       [1119.09  , 1121.53  , 1119.09  , 1121.53  ],
       [1119.09  , 1119.09  , 1119.    , 1119.    ],
       [1119.    , 1119.    , 1119.    , 1119.    ]])

# Load technical features

A library: https://technical-analysis-library-in-python.readthedocs.io/en/latest/

### Features used in the literature:

* Stochastic K - Implemented
* Stochastic D - Implemented
* Slow Stochastic D - Implemented
* Momentum - Same as difference
* ROC - Implemented
* Williams % R - Implemented
* A/D Oscillator
* Disparity 5 - Implemented
* Disparity 10 - Implemented
* Price Oscillator - (detrended) - Implemented
* Commodity Channel Index - Implemented
* RSI - Impliemented

Formulas: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=876544

* Moving Average - Implemented
* Bias
* Exponential Moving Average - Implemented
* Difference - Same af Momentum
* True Range - (Average) - Implemented
* 

Formulas: https://www.sciencedirect.com/science/article/pii/S0957417407001819?via%3Dihub

#### Other Technical Features
* Moving Average Convergence Divergence (MACD) - Implemented

**Non-classical technical features**

* Bid/Ask prices of top of book
* Spread and mid price based on top og book
* Price derivatives

Formulas: https://www.tandfonline.com/doi/full/10.1080/14697688.2015.1032546?instName=UCL+%28University+College+London%29

In [8]:
candles[:5]

array([[1124.46  , 1124.46  , 1124.2033, 1124.2033],
       [1124.62  , 1125.74  , 1124.51  , 1125.74  ],
       [1125.74  , 1125.74  , 1124.2   , 1124.2   ],
       [1124.2   , 1126.94  , 1124.2   , 1124.2   ],
       [1124.2   , 1124.2   , 1124.2   , 1124.2   ]])

In [9]:
FEATURE_LAGS = 1
features = generateFeatures_final(data = candles, 
                                   listOfFeatures = ['pastobs',
                                                      'stok',
#                                                       'stod',
#                                                       'sstod',
#                                                       'wilr',
#                                                       'roc',
#                                                       'rsi',
#                                                       'atr',
#                                                       'cci',
#                                                       'dpo',
#                                                       'sma',
#                                                       'ema',
#                                                       'macd',
#                                                       'dis5',
#                                                       'dis10'
                                                    ], 
                                   feature_lags = FEATURE_LAGS) # 0 for no lags, 1 for 1 lag (returns both raw & first lag)
features[:20] 

,open_lag0,high_lag0,low_lag0,close_lag0,stok_lag0,open_lag1,high_lag1,low_lag1,close_lag1,stok_lag1
0,1124.620,1125.74,1124.510,1125.740,NaN,1124.460,1124.46,1124.2033,1124.2033,NaN
1,1125.740,1125.74,1124.200,1124.200,NaN,1124.620,1125.74,1124.5100,1125.7400,NaN
2,1124.200,1126.94,1124.200,1124.200,NaN,1125.740,1125.74,1124.2000,1124.2000,NaN
3,1124.200,1124.20,1124.200,1124.200,NaN,1124.200,1126.94,1124.2000,1124.2000,NaN
4,1124.200,1124.52,1124.200,1124.200,NaN,1124.200,1124.20,1124.2000,1124.2000,NaN
5,1124.605,1130.00,1124.200,1126.620,NaN,1124.200,1124.52,1124.2000,1124.2000,NaN
6,1130.000,1130.00,1124.210,1124.210,NaN,1124.605,1130.00,1124.2000,1126.6200,NaN
7,1130.000,1130.00,1126.100,1130.000,NaN,1130.000,1130.00,1124.2100,1124.2100,NaN
8,1127.000,1127.00,1125.082,1125.082,NaN,1130.000,1130.00,1126.1000,1130.0000,NaN
9,1125.790,1125.79,1125.790,1125.790,NaN,1127.000,1127.00,1125.0820,1125.0820,NaN


In [10]:
features[-20:]

,open_lag0,high_lag0,low_lag0,close_lag0,stok_lag0,open_lag1,high_lag1,low_lag1,close_lag1,stok_lag1
880,1121.1100,1121.1100,1121.1100,1121.1100,79.357143,1121.1100,1122.4300,1121.0000,1122.4300,88.785714
881,1121.1100,1121.1100,1121.1100,1121.1100,79.357143,1121.1100,1121.1100,1121.1100,1121.1100,79.357143
882,1120.8400,1121.1100,1120.8400,1121.1100,85.461538,1121.1100,1121.1100,1121.1100,1121.1100,79.357143
883,1121.1100,1121.5300,1121.1100,1121.1100,85.461538,1120.8400,1121.1100,1120.8400,1121.1100,85.461538
884,1120.8400,1121.1100,1120.8400,1121.1100,85.461538,1121.1100,1121.5300,1121.1100,1121.1100,85.461538
885,1120.8400,1121.5300,1120.8400,1121.5300,88.692308,1120.8400,1121.1100,1120.8400,1121.1100,85.461538
886,1122.0550,1122.0550,1122.0550,1122.0550,92.730769,1120.8400,1121.5300,1120.8400,1121.5300,88.692308
887,1122.0550,1122.0550,1122.0550,1122.0550,92.730769,1122.0550,1122.0550,1122.0550,1122.0550,92.730769
888,1121.1100,1121.1100,1121.1100,1121.1100,16.981132,1122.0550,1122.0550,1122.0550,1122.0550,92.730769
889,1121.1100,1121.1100,1121.1100,1121.1100,16.981132,1121.1100,1121.1100,1121.1100,1121.1100,16.981132


# Preprocessing of the features 
- Make sure burn-in is removed correctly (dependent on using sample=='full' or sample=='stable')
 * It should be daily burn-in for sample=='stable' and only-once burn-in for sample=='full'
 * We note here that sample=='full' is slightly misleading, as it's not the full data sample but only full market open [9, 16.5]
- Min/max, std, norm, difference-to-close
- Be aware that not all preprocessing approaches make sense for all features (e.g. difference-to-close is only meant for features that describe a price)


In [11]:
# first extract labels
#labels = extract_labels(data = candles, classes = 5, group_style = 'equal')

In [205]:
def extract_labels(data = '', classes = 5, group_style = 'equal'):
    
   # returns = ((data.T[-1][1:]/data.T[-1][0:-1])-1)*100
    returns = ((data[1:, -1] / data[:-1, -1])-1)*100
    
    labels = np.zeros(returns.shape[0])

    if group_style == 'equal':
        thresholdsMin = [np.array_split(np.sort(returns),classes)[i].min() for i in np.arange(classes)]
        thresholdsMax = [np.array_split(np.sort(returns),classes)[i].max() for i in np.arange(classes)]    
    elif group_style != 'equal':
        raise ValueError(f'group_style {group_style} not implemented')
    
    for i in np.arange(classes):
        if i == 0:
            labels[(returns <= thresholdsMax[i])] = i

        elif i == (classes-1):
            labels[(returns >= thresholdsMin[i])] = i

        else:
            labels[(returns >= thresholdsMin[i]) & (returns<=thresholdsMax[i])] = i  
            
    return labels, returns, [thresholdsMin, thresholdsMax]

group_style = 'equal'
classes = 5

#returns = ((candles.T[-1][1:]/candles.T[-1][0:-1])-1)*100
returns = ((candles[1:, -1] / candles[:-1, -1])-1)*100

labels = np.zeros(returns.shape[0])

if group_style == 'equal':
    thresholdsMin = [np.array_split(np.sort(returns),classes)[i].min() for i in np.arange(classes)]
    thresholdsMax = [np.array_split(np.sort(returns),classes)[i].max() for i in np.arange(classes)]    
elif group_style != 'equal':
    raise ValueError(f'group_style {group_style} not implemented')

for i in np.arange(classes):
    if i == 0:
        labels[(returns <= thresholdsMax[i])] = i

    elif i == (classes-1):
        labels[(returns >= thresholdsMin[i])] = i

    else:
        labels[(returns >= thresholdsMin[i]) & (returns<=thresholdsMax[i])] = i  

#return labels



In [206]:
candles

array([[1124.46  , 1124.46  , 1124.2033, 1124.2033],
       [1124.62  , 1125.74  , 1124.51  , 1125.74  ],
       [1125.74  , 1125.74  , 1124.2   , 1124.2   ],
       ...,
       [1119.09  , 1121.53  , 1119.09  , 1121.53  ],
       [1119.09  , 1119.09  , 1119.    , 1119.    ],
       [1119.    , 1119.    , 1119.    , 1119.    ]])

In [207]:
thresholdsMin, thresholdsMax

([-0.5728085977076858,
  -0.10187603155042968,
  -0.023824936167138322,
  0.02411821512358081,
  0.09681449094316452],
 [-0.1020152485950665,
  -0.023979324493550447,
  0.0237205148947206,
  0.09677288300853348,
  0.6751130411425166])

In [208]:
returns

array([ 1.36692358e-01, -1.36798906e-01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  2.15264188e-01, -2.13914186e-01,  5.15028331e-01,
       -4.35221239e-01,  6.29287465e-02, -3.37540749e-02,  0.00000000e+00,
        0.00000000e+00,  4.07851361e-01, -1.96460177e-01,  1.96846903e-01,
       -4.43362832e-01, -7.02228464e-02,  7.02721936e-02, -8.80007822e-02,
        1.77935943e-01, -1.77619893e-02, -1.59886303e-01,  3.82562278e-02,
        3.82415980e-02,  1.01345945e-01, -2.66429840e-03, -2.39793245e-02,
       -3.05587634e-01,  3.26127635e-01, -1.63377505e-01,  3.30003056e-01,
       -4.47654262e-01, -9.23097696e-02, -4.32823247e-01,  6.75113041e-01,
        3.04449524e-01, -1.00477632e-01, -2.32541627e-01,  4.67590698e-02,
        1.04313980e-01,  2.67641725e-02, -7.90277605e-02, -1.03993165e-01,
       -1.67562278e-01, -2.90219394e-01,  5.00509447e-02,  1.78663951e-03,
        5.00250125e-02,  1.17857143e-01, -6.50750901e-02, -1.80253211e-01,
        1.92915886e-01, -

In [209]:
labels

array([4., 0., 2., 2., 2., 4., 0., 4., 0., 3., 1., 2., 2., 4., 0., 4., 0.,
       1., 3., 1., 4., 2., 0., 3., 3., 4., 2., 1., 0., 4., 0., 4., 0., 1.,
       0., 4., 4., 1., 0., 3., 4., 3., 1., 0., 0., 0., 3., 2., 3., 4., 1.,
       0., 4., 0., 3., 3., 1., 0., 3., 3., 0., 3., 3., 1., 4., 4., 4., 4.,
       1., 2., 3., 2., 4., 4., 2., 2., 0., 3., 0., 3., 2., 1., 1., 0., 2.,
       2., 3., 0., 4., 2., 4., 3., 3., 2., 3., 2., 2., 3., 1., 1., 1., 4.,
       1., 1., 4., 1., 1., 0., 3., 1., 1., 3., 3., 0., 1., 2., 4., 1., 4.,
       1., 2., 1., 4., 3., 3., 2., 1., 2., 2., 1., 3., 0., 0., 4., 0., 3.,
       4., 3., 2., 3., 0., 1., 0., 1., 2., 1., 3., 1., 1., 1., 3., 3., 1.,
       3., 3., 0., 4., 1., 0., 1., 1., 1., 3., 4., 1., 0., 2., 2., 2., 4.,
       3., 1., 3., 2., 3., 1., 4., 1., 0., 2., 0., 1., 3., 2., 4., 1., 0.,
       1., 2., 4., 2., 0., 4., 4., 1., 1., 1., 2., 1., 3., 3., 1., 3., 1.,
       3., 0., 1., 3., 2., 2., 2., 3., 0., 3., 1., 1., 2., 0., 0., 2., 1.,
       0., 3., 4., 2., 1.

In [210]:
SAFE_BURN_IN = False # True for restarting burn-in each day, False for continuing from yesterday's close (DATA_SAMPLE = 'stable' requires SAFE_BURN_IN = True )

if (DATA_SAMPLE == 'stable') & (not SAFE_BURN_IN):
    raise ValueError(f'DATA_SAMPLE = "{DATA_SAMPLE}" & SAFE_BURN_IN = "{SAFE_BURN_IN}" not allowed. Burn-in needs a restart with DATA_SAMPLE = "{DATA_SAMPLE}" as we skip 30 minutes')

# # postponing this for later
# to do: allow burn-in to happen every day (we probably need to extract the MultiIndex from candles to do this in a nice vectorized way)
    # 
# if SAFE_BURN_IN:
    
#     # for each day ...
    
#     # get first index that has no NaNs (the sum checks for True across columns, we look for sum == 0 and where that is first True)
#     burned_in_idx = np.where((np.sum(np.isnan(features.values), axis=1) == 0) == True)[0][0]
    
#     # slice away the observations used for burn-in
#     burned_in_features = features.iloc[burned_in_idx:, :] # features[burned_in_idx:] latter is sligthly faster but maybe not as precise
    
#     # check if there are remaining NaNs are burn-in (means error)
#     remaining_nans = np.where(np.isnan(burned_in_features.values))[0].size
#     if remaining_nans > 0:
#         raise ValueError('Had NaN in burned_in_features after burn-in')
        
prediction_horizon = 1
    
if not SAFE_BURN_IN:
    assert DATA_SAMPLE == 'full'
    # we assume data_sample is full and that we can continue features from yesterday's values.
    # that we have a single burn-in at the beginning and that's it
    
    # get first index that has no NaNs (the sum checks for True across columns, we look for sum == 0 and where that is first True)
    burned_in_idx = np.where((np.sum(np.isnan(features.values), axis=1) == 0) == True)[0][0]
    
    # calculate end-point cut-off to match with labels
    end_point_cut = max(prediction_horizon, FEATURE_LAGS + 1)# + 1 #prediction_horizon + (FEATURE_LAGS - 2)
    
    # slice away the observations used for burn-in (taking off 1 at the end to match with labels [slice off "prediction_horizon"])
    burned_in_features = features.iloc[burned_in_idx : -end_point_cut, :].reset_index(drop=True) # features[burned_in_idx:] latter is sligthly faster but maybe not as precise
    
    # slice away the burned-in indices from labels
    #labels, _, _ = extract_labels(data = candles[burned_in_idx:, :], classes = 5, group_style = 'equal')
    labels, returns, thresholds = extract_labels(data = candles[burned_in_idx + FEATURE_LAGS : , :], classes = 5, group_style = 'equal')
    
    # check if there are remaining NaNs are burn-in (means error)
    remaining_nans = np.where(np.isnan(burned_in_features.values))[0].size
    if remaining_nans > 0:
        raise ValueError('Had NaN in burned_in_features after burn-in')    

In [211]:
burned_in_features

,open_lag0,high_lag0,low_lag0,close_lag0,stok_lag0,open_lag1,high_lag1,low_lag1,close_lag1,stok_lag1,open_lag2,high_lag2,low_lag2,close_lag2,stok_lag2
0,1127.7500,1127.7800,1127.7500,1127.7800,61.724138,1130.0000,1130.0000,1130.0000,1130.0000,100.000000,1125.4100,1125.4100,1125.4100,1125.4100,20.862069
1,1126.0100,1130.0000,1125.2100,1130.0000,100.000000,1127.7500,1127.7800,1127.7500,1127.7800,61.724138,1130.0000,1130.0000,1130.0000,1130.0000,100.000000
2,1130.0000,1130.0000,1124.9900,1124.9900,13.620690,1126.0100,1130.0000,1125.2100,1130.0000,100.000000,1127.7500,1127.7800,1127.7500,1127.7800,61.724138
3,1124.9900,1125.8900,1124.2000,1124.2000,0.000000,1130.0000,1130.0000,1124.9900,1124.9900,13.620690,1126.0100,1130.0000,1125.2100,1130.0000,100.000000
4,1124.2000,1125.8900,1124.2000,1124.9900,13.620690,1124.9900,1125.8900,1124.2000,1124.2000,0.000000,1130.0000,1130.0000,1124.9900,1124.9900,13.620690
5,1125.2500,1125.9000,1124.0000,1124.0000,0.000000,1124.2000,1125.8900,1124.2000,1124.9900,13.620690,1124.9900,1125.8900,1124.2000,1124.2000,0.000000
6,1124.2000,1126.0000,1124.0000,1126.0000,33.333333,1125.2500,1125.9000,1124.0000,1124.0000,0.000000,1124.2000,1125.8900,1124.2000,1124.9900,13.620690
7,1125.0000,1125.8000,1124.2000,1125.8000,30.000000,1124.2000,1126.0000,1124.0000,1126.0000,33.333333,1125.2500,1125.9000,1124.0000,1124.0000,0.000000
8,1125.7000,1125.7900,1124.0000,1124.0000,0.000000,1125.0000,1125.8000,1124.2000,1125.8000,30.000000,1124.2000,1126.0000,1124.0000,1126.0000,33.333333
9,1124.0000,1124.4300,1124.0000,1124.4300,7.166667,1125.7000,1125.7900,1124.0000,1124.0000,0.000000,1125.0000,1125.8000,1124.2000,1125.8000,30.000000


In [212]:
labels, returns, thresholds

(array([4., 0., 1., 3., 1., 4., 2., 0., 3., 3., 4., 2., 2., 0., 4., 0., 4.,
        0., 1., 0., 4., 4., 1., 0., 3., 4., 3., 1., 0., 0., 0., 3., 2., 3.,
        4., 1., 0., 4., 0., 3., 3., 1., 0., 3., 3., 0., 3., 3., 1., 4., 4.,
        4., 4., 1., 2., 3., 2., 4., 4., 2., 2., 0., 3., 0., 3., 2., 1., 1.,
        0., 2., 2., 3., 0., 4., 2., 4., 3., 3., 2., 3., 2., 2., 3., 1., 1.,
        1., 4., 1., 1., 4., 1., 1., 0., 3., 1., 1., 3., 3., 0., 0., 2., 4.,
        1., 4., 1., 2., 1., 4., 3., 3., 2., 1., 2., 2., 1., 3., 0., 0., 4.,
        0., 3., 4., 3., 2., 3., 0., 1., 0., 1., 2., 1., 3., 1., 1., 1., 3.,
        3., 1., 3., 3., 0., 4., 1., 0., 1., 1., 1., 3., 4., 1., 0., 2., 2.,
        2., 4., 3., 1., 3., 2., 3., 1., 4., 1., 0., 2., 0., 1., 3., 2., 4.,
        1., 0., 1., 2., 4., 2., 0., 4., 4., 1., 1., 1., 2., 1., 3., 3., 1.,
        3., 1., 3., 0., 1., 3., 2., 2., 2., 3., 0., 3., 1., 1., 2., 0., 0.,
        2., 1., 0., 3., 4., 2., 1., 4., 0., 3., 3., 4., 3., 1., 4., 2., 1.,
        1., 

In [213]:
labels.shape, returns.shape, features.shape, burned_in_features.shape

((884,), (884,), (900, 15), (884, 15))

In [214]:
features[0:20]

,open_lag0,high_lag0,low_lag0,close_lag0,stok_lag0,open_lag1,high_lag1,low_lag1,close_lag1,stok_lag1,open_lag2,high_lag2,low_lag2,close_lag2,stok_lag2
0,1125.740,1125.74,1124.200,1124.200,NaN,1124.620,1125.74,1124.510,1125.740,NaN,1124.460,1124.46,1124.2033,1124.2033,NaN
1,1124.200,1126.94,1124.200,1124.200,NaN,1125.740,1125.74,1124.200,1124.200,NaN,1124.620,1125.74,1124.5100,1125.7400,NaN
2,1124.200,1124.20,1124.200,1124.200,NaN,1124.200,1126.94,1124.200,1124.200,NaN,1125.740,1125.74,1124.2000,1124.2000,NaN
3,1124.200,1124.52,1124.200,1124.200,NaN,1124.200,1124.20,1124.200,1124.200,NaN,1124.200,1126.94,1124.2000,1124.2000,NaN
4,1124.605,1130.00,1124.200,1126.620,NaN,1124.200,1124.52,1124.200,1124.200,NaN,1124.200,1124.20,1124.2000,1124.2000,NaN
5,1130.000,1130.00,1124.210,1124.210,NaN,1124.605,1130.00,1124.200,1126.620,NaN,1124.200,1124.52,1124.2000,1124.2000,NaN
6,1130.000,1130.00,1126.100,1130.000,NaN,1130.000,1130.00,1124.210,1124.210,NaN,1124.605,1130.00,1124.2000,1126.6200,NaN
7,1127.000,1127.00,1125.082,1125.082,NaN,1130.000,1130.00,1126.100,1130.000,NaN,1130.000,1130.00,1124.2100,1124.2100,NaN
8,1125.790,1125.79,1125.790,1125.790,NaN,1127.000,1127.00,1125.082,1125.082,NaN,1130.000,1130.00,1126.1000,1130.0000,NaN
9,1125.410,1125.41,1125.410,1125.410,NaN,1125.790,1125.79,1125.790,1125.790,NaN,1127.000,1127.00,1125.0820,1125.0820,NaN


In [215]:
candles

array([[1124.46  , 1124.46  , 1124.2033, 1124.2033],
       [1124.62  , 1125.74  , 1124.51  , 1125.74  ],
       [1125.74  , 1125.74  , 1124.2   , 1124.2   ],
       ...,
       [1119.09  , 1121.53  , 1119.09  , 1121.53  ],
       [1119.09  , 1119.09  , 1119.    , 1119.    ],
       [1119.    , 1119.    , 1119.    , 1119.    ]])

In [216]:
burned_in_features[:3]

,open_lag0,high_lag0,low_lag0,close_lag0,stok_lag0,open_lag1,high_lag1,low_lag1,close_lag1,stok_lag1,open_lag2,high_lag2,low_lag2,close_lag2,stok_lag2
0,1127.75,1127.78,1127.75,1127.78,61.724138,1130.00,1130.00,1130.00,1130.00,100.000000,1125.41,1125.41,1125.41,1125.41,20.862069
1,1126.01,1130.00,1125.21,1130.00,100.000000,1127.75,1127.78,1127.75,1127.78,61.724138,1130.00,1130.00,1130.00,1130.00,100.000000
2,1130.00,1130.00,1124.99,1124.99,13.620690,1126.01,1130.00,1125.21,1130.00,100.000000,1127.75,1127.78,1127.75,1127.78,61.724138


In [217]:
# candles[burned_in_idx + FEATURE_LAGS: -(end_point_cut-1), :][:3]
candles[burned_in_idx + FEATURE_LAGS: -(end_point_cut), :][:3]

array([[1127.75, 1127.78, 1127.75, 1127.78],
       [1126.01, 1130.  , 1125.21, 1130.  ],
       [1130.  , 1130.  , 1124.99, 1124.99]])

In [218]:
 labels[:3], returns[:3]

(array([4., 0., 1.]), array([ 0.1968469 , -0.44336283, -0.07022285]))

In [219]:
# So the increase of 0.40785136 % matches index 13 to 14 (0 to 1 of burned_in_features vector), so this is fine (0.40785136 % should be the return used in label for index 13)
# Means that most likely only the end-point has to be removed, lets check:

In [220]:
# example of X, y:
burned_in_features[:1], labels[:1], returns[:1]

(   open_lag0  high_lag0  low_lag0  close_lag0  stok_lag0  open_lag1  \
 0    1127.75    1127.78   1127.75     1127.78  61.724138     1130.0   
 
    high_lag1  low_lag1  close_lag1  stok_lag1  open_lag2  high_lag2  low_lag2  \
 0     1130.0    1130.0      1130.0      100.0    1125.41    1125.41   1125.41   
 
    close_lag2  stok_lag2  
 0     1125.41  20.862069  , array([4.]), array([0.1968469]))

In [221]:
# Check the end-point
features[-6:]

,open_lag0,high_lag0,low_lag0,close_lag0,stok_lag0,open_lag1,high_lag1,low_lag1,close_lag1,stok_lag1,open_lag2,high_lag2,low_lag2,close_lag2,stok_lag2
894,1119.09,1119.09,1119.09,1119.09,0.000000,1121.5267,1121.5267,1121.5267,1121.5267,82.182125,1119.0900,1122.0000,1119.0900,1122.0000,98.145025
895,1119.09,1121.53,1119.09,1121.53,82.293423,1119.0900,1119.0900,1119.0900,1119.0900,0.000000,1121.5267,1121.5267,1121.5267,1121.5267,82.182125
896,1119.09,1119.09,1119.00,1119.00,0.000000,1119.0900,1121.5300,1119.0900,1121.5300,82.293423,1119.0900,1119.0900,1119.0900,1119.0900,0.000000
897,1119.00,1119.00,1119.00,1119.00,0.000000,1119.0900,1119.0900,1119.0000,1119.0000,0.000000,1119.0900,1121.5300,1119.0900,1121.5300,82.293423
898,NaN,NaN,NaN,NaN,NaN,1119.0000,1119.0000,1119.0000,1119.0000,0.000000,1119.0900,1119.0900,1119.0000,1119.0000,0.000000
899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1119.0000,1119.0000,1119.0000,1119.0000,0.000000


In [222]:
burned_in_features[-6:]

,open_lag0,high_lag0,low_lag0,close_lag0,stok_lag0,open_lag1,high_lag1,low_lag1,close_lag1,stok_lag1,open_lag2,high_lag2,low_lag2,close_lag2,stok_lag2
878,1121.5300,1121.5300,1121.5300,1121.5300,73.053892,1119.4500,1121.5300,1119.0900,1121.5300,73.053892,1121.1100,1121.1100,1121.1100,1121.1100,16.981132
879,1119.0900,1122.0000,1119.0900,1122.0000,98.145025,1121.5300,1121.5300,1121.5300,1121.5300,73.053892,1119.4500,1121.5300,1119.0900,1121.5300,73.053892
880,1121.5267,1121.5267,1121.5267,1121.5267,82.182125,1119.0900,1122.0000,1119.0900,1122.0000,98.145025,1121.5300,1121.5300,1121.5300,1121.5300,73.053892
881,1119.0900,1119.0900,1119.0900,1119.0900,0.000000,1121.5267,1121.5267,1121.5267,1121.5267,82.182125,1119.0900,1122.0000,1119.0900,1122.0000,98.145025
882,1119.0900,1121.5300,1119.0900,1121.5300,82.293423,1119.0900,1119.0900,1119.0900,1119.0900,0.000000,1121.5267,1121.5267,1121.5267,1121.5267,82.182125
883,1119.0900,1119.0900,1119.0000,1119.0000,0.000000,1119.0900,1121.5300,1119.0900,1121.5300,82.293423,1119.0900,1119.0900,1119.0900,1119.0900,0.000000


In [224]:
candles[burned_in_idx + FEATURE_LAGS: -(end_point_cut), :][-6:]

array([[1121.11  , 1121.11  , 1121.11  , 1121.11  ],
       [1119.45  , 1121.53  , 1119.09  , 1121.53  ],
       [1121.53  , 1121.53  , 1121.53  , 1121.53  ],
       [1119.09  , 1122.    , 1119.09  , 1122.    ],
       [1121.5267, 1121.5267, 1121.5267, 1121.5267],
       [1119.09  , 1119.09  , 1119.09  , 1119.09  ]])

In [225]:
 labels[-6:], returns[-6:]

(array([3., 1., 0., 4., 0., 2.]),
 array([ 0.04190704, -0.0421836 , -0.21726634,  0.2180343 , -0.22558469,
         0.        ]))

In [226]:
features[-6:]

,open_lag0,high_lag0,low_lag0,close_lag0,stok_lag0,open_lag1,high_lag1,low_lag1,close_lag1,stok_lag1,open_lag2,high_lag2,low_lag2,close_lag2,stok_lag2
894,1119.09,1119.09,1119.09,1119.09,0.000000,1121.5267,1121.5267,1121.5267,1121.5267,82.182125,1119.0900,1122.0000,1119.0900,1122.0000,98.145025
895,1119.09,1121.53,1119.09,1121.53,82.293423,1119.0900,1119.0900,1119.0900,1119.0900,0.000000,1121.5267,1121.5267,1121.5267,1121.5267,82.182125
896,1119.09,1119.09,1119.00,1119.00,0.000000,1119.0900,1121.5300,1119.0900,1121.5300,82.293423,1119.0900,1119.0900,1119.0900,1119.0900,0.000000
897,1119.00,1119.00,1119.00,1119.00,0.000000,1119.0900,1119.0900,1119.0000,1119.0000,0.000000,1119.0900,1121.5300,1119.0900,1121.5300,82.293423
898,NaN,NaN,NaN,NaN,NaN,1119.0000,1119.0000,1119.0000,1119.0000,0.000000,1119.0900,1119.0900,1119.0000,1119.0000,0.000000
899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1119.0000,1119.0000,1119.0000,1119.0000,0.000000


In [176]:
# The decrease of -0.22558469 % matches index 895 to 896 (882 to 883 of burned_in_features vector)
# Means that the feature matrix has one row too many as expected (index 899 has no subsequent close price) so no label
# we solve this by slicing off 1 row at the end.
# note that we should actually, in the general case, slice off "prediction_horizon" (equal to 1 here) at the end of the feature matrix

In [177]:
# example of X, y:
burned_in_features[-2:], labels[-2:], returns[-2:]

(        open     high      low    close       stok
 884  1119.09  1121.53  1119.09  1121.53  82.293423
 885  1119.09  1119.09  1119.00  1119.00   0.000000,
 array([0., 2.]),
 array([-0.22558469,  0.        ]))

# Fix that features and labels are not aligned when lag = 0

In [15]:
# SAFE_BURN_IN = False # True for restarting burn-in each day, False for continuing from yesterday's close (DATA_SAMPLE = 'stable' requires SAFE_BURN_IN = True )

# if (DATA_SAMPLE == 'stable') & (not SAFE_BURN_IN):
#     raise ValueError(f'DATA_SAMPLE = "{DATA_SAMPLE}" & SAFE_BURN_IN = "{SAFE_BURN_IN}" not allowed. Burn-in needs a restart with DATA_SAMPLE = "{DATA_SAMPLE}" as we skip 30 minutes')

# # # postponing this for later
# # to do: allow burn-in to happen every day (we probably need to extract the MultiIndex from candles to do this in a nice vectorized way)
#     # 
# # if SAFE_BURN_IN:
    
# #     # for each day ...
    
# #     # get first index that has no NaNs (the sum checks for True across columns, we look for sum == 0 and where that is first True)
# #     burned_in_idx = np.where((np.sum(np.isnan(features.values), axis=1) == 0) == True)[0][0]
    
# #     # slice away the observations used for burn-in
# #     burned_in_features = features.iloc[burned_in_idx:, :] # features[burned_in_idx:] latter is sligthly faster but maybe not as precise
    
# #     # check if there are remaining NaNs are burn-in (means error)
# #     remaining_nans = np.where(np.isnan(burned_in_features.values))[0].size
# #     if remaining_nans > 0:
# #         raise ValueError('Had NaN in burned_in_features after burn-in')
        
# prediction_horizon = 1
    
# if not SAFE_BURN_IN:
#     assert DATA_SAMPLE == 'full'
#     # we assume data_sample is full and that we can continue features from yesterday's values.
#     # that we have a single burn-in at the beginning and that's it
    
#     # get first index that has no NaNs (the sum checks for True across columns, we look for sum == 0 and where that is first True)
#     burned_in_idx = np.where((np.sum(np.isnan(features.values), axis=1) == 0) == True)[0][0]
    
#     # calculate end-point cut-off to match with labels
#     end_point_cut = max(prediction_horizon, FEATURE_LAGS) + 1 #prediction_horizon + (FEATURE_LAGS - 2)
    
#     # slice away the observations used for burn-in (taking off 1 at the end to match with labels [slice off "prediction_horizon"])
#     burned_in_features = features.iloc[burned_in_idx : -end_point_cut, :].reset_index(drop=True) # features[burned_in_idx:] latter is sligthly faster but maybe not as precise
    
#     # slice away the burned-in indices from labels
#     #labels, _, _ = extract_labels(data = candles[burned_in_idx:, :], classes = 5, group_style = 'equal')
#     labels, returns, thresholds = extract_labels(data = candles[burned_in_idx + FEATURE_LAGS : , :], classes = 5, group_style = 'equal')
    
#     # check if there are remaining NaNs are burn-in (means error)
#     remaining_nans = np.where(np.isnan(burned_in_features.values))[0].size
#     if remaining_nans > 0:
#         raise ValueError('Had NaN in burned_in_features after burn-in')    

def align_features_and_labels(candles, prediction_horizon, features, n_feature_lags, n_classes,
                              safe_burn_in = False, data_sample = 'full'):
    
    if not safe_burn_in:
        assert data_sample == 'full'
        # we assume data_sample is full and that we can continue features from yesterday's values.
        # that we have a single burn-in at the beginning and that's it

        # get first index that has no NaNs (the sum checks for True across columns, we look for sum == 0 and where that is first True)
        burned_in_idx = np.where((np.sum(np.isnan(features.values), axis=1) == 0) == True)[0][0]

        # calculate end-point cut-off to match with labels
        end_point_cut = max(prediction_horizon, n_feature_lags + 1)# + 1 #prediction_horizon + (FEATURE_LAGS - 2)

        # slice away the observations used for burn-in (taking off 1 at the end to match with labels [slice off "prediction_horizon"])
        burned_in_features = features.iloc[burned_in_idx : -end_point_cut, :].reset_index(drop=True) # features[burned_in_idx:] latter is sligthly faster but maybe not as precise

        # slice away the burned-in indices from labels
        labels, _, _ = extract_labels(data = candles[burned_in_idx + n_feature_lags :, :], 
                                      classes = n_classes, group_style = 'equal')
        # labels, returns, thresholds = extract_labels(data = candles[burned_in_idx + n_feature_lags : , :], 
        #                                             classes = n_classes, group_style = 'equal')

        # check if there are remaining NaNs are burn-in (means error)
        remaining_nans = np.where(np.isnan(burned_in_features.values))[0].size
        if remaining_nans > 0:
            raise ValueError('Had NaN in burned_in_features after burn-in')   
            
    return burned_in_features, labels # call the function as X, y = align_features_and_labels(.) if you like

In [ ]:
from utils.preprocessing_features_and_labels import align_features_and_labels as test_align

In [19]:


FEATURE_LAGS = 0
features = generateFeatures_final(data = candles, 
                                   listOfFeatures = ['pastobs',
                                                      'stok',
#                                                       'stod',
#                                                       'sstod',
#                                                       'wilr',
#                                                       'roc',
#                                                       'rsi',
#                                                       'atr',
#                                                       'cci',
#                                                       'dpo',
#                                                       'sma',
#                                                       'ema',
#                                                       'macd',
#                                                       'dis5',
#                                                       'dis10'
                                                    ], 
                                   feature_lags = FEATURE_LAGS) # 0 for no lags, 1 for 1 lag (returns both raw & first lag) 

n_feature_lags = FEATURE_LAGS
n_classes = 5

X, y = test_align(candles = candles, 
                                 prediction_horizon = 1, 
                                 features = features, 
                                 n_feature_lags = n_feature_lags, 
                                 n_classes = n_classes, # 5,
                                 safe_burn_in = False, 
                                 data_sample = 'full')
print(X.shape, y.shape)

X, y = align_features_and_labels(candles = candles, 
                                 prediction_horizon = 1, 
                                 features = features, 
                                 n_feature_lags = n_feature_lags, 
                                 n_classes = n_classes, # 5,
                                 safe_burn_in = False, 
                                 data_sample = 'full')
print(X.shape, y.shape)



(886, 5) (886,)
(886, 5) (886,)


# Drafts and ad hoc timings

In [58]:
#features.isnull().sum(axis=1) == 0
%timeit np.where(np.isnan(features.values))[0][-1] # 5 times faster than with .values
%timeit np.where(np.isnan(features))[0][-1]
%timeit np.where(features.isnull())[0][-1]
#np.where(np.isnan(x))

47.2 µs ± 4.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
234 µs ± 26.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
239 µs ± 27.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [60]:
np.where(np.isnan(features.values))

(array([ 0,  0,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
        13], dtype=int64),
 array([0, 1, 2, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=int64))

In [66]:
%timeit np.where((features.isnull().sum(axis=1) == 0)==True)[0][0]
%timeit np.where((np.sum(np.isnan(features.values), axis=1) == 0) == True)[0][0]

1.05 ms ± 8.31 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
52.3 µs ± 483 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [67]:
np.where((np.sum(np.isnan(features.values), axis=1) == 0)==True)[0][0]

14

In [68]:
burned_in_idx = np.where((np.sum(np.isnan(features.values), axis=1) == 0) == True)[0][0]

In [71]:
%timeit features[burned_in_idx:]
%timeit features.iloc[burned_in_idx:, :]

114 µs ± 5.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
143 µs ± 581 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [73]:
features.iloc[burned_in_idx:, :][:10]

,open,high,low,close,stok
14,1125.79,1125.79,1125.79,1125.79,51.405622
15,1127.75,1127.78,1127.75,1127.78,100.000000
16,1126.01,1126.01,1125.21,1125.21,0.000000
17,1126.00,1127.00,1126.00,1127.00,69.649805
18,1124.99,1125.89,1124.99,1125.89,32.258065
19,1125.89,1125.89,1124.99,1124.99,0.000000
20,1125.90,1125.90,1124.00,1124.00,0.000000
21,1124.00,1124.00,1124.00,1124.00,0.000000
22,1124.20,1124.20,1124.20,1124.20,5.291005
23,1125.70,1125.79,1124.00,1124.00,0.000000


In [84]:
# Make sure there are no NaNs (this would be mean holes in candlesticks and something else wrong)
np.where(np.isnan(features.iloc[burned_in_idx:, :].values))[0].size

0